In [ ]:
from model import *
from data import *
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint 

In [ ]:
data_gen_args = dict(rotation_range=0.2,
                    width_shift_range=0.05,
                    height_shift_range=0.05,
                    shear_range=0.05,
                    zoom_range=0.05,
                    horizontal_flip=True,
                    vertical_flip=True,
                    fill_mode='nearest')
myGene = trainGenerator(2,'dataset','images','labels',data_gen_args,save_to_dir = None)
model = satellite_unet()
model_checkpoint = ModelCheckpoint('sat_unet.hdf5', monitor='loss',verbose=1, save_best_only=True)
model.fit_generator(myGene,steps_per_epoch=2000,epochs=5,callbacks=[model_checkpoint])